In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00


In [2]:
api_key = "******************************************"

In [4]:
pip install langchain langchain_community pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

### Analogy

Think of the index like a library catalog:

Books = Documents or items you want to search.
Book Summaries = Embeddings or vectors (representing the content of the books).
Catalog = The index, which organizes the books and their summaries for quick lookup.
When you search for a topic, the catalog helps you find relevant books efficiently.

### In Summary

Creating an index is essential because:

It organizes your data (vectors + metadata).
It enables fast, scalable, and accurate searches.
It supports advanced use cases like semantic search, hybrid search, and recommendations.

In [6]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
# initialize the Pinecone Client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,  #dimension of dense vector since hugging face technique gives vector of this size
        metric="dotproduct",
        spec=ServerlessSpec(cloud = 'aws', region = "us-east-1")
        )

# ServerlessSpec - Specifies that this index will run in a serverless mode (i.e., automatically scaled based on usage).

In [7]:
index = pc.Index(index_name)
index

In [16]:
# vector embeddings and sparse matrix
import os
# from dotenv import load_dotenv
# load_dotenv()

os.environ["HF_TOKEN"] = "***********"

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() #uses TF-IDF by default
bm25_encoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
senetences = [
    "Artificial intelligence is transforming industries.",
    "Pinecone provides vector database solutions.",
    "Hybrid search combines semantic and keyword search."
]

In [20]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
# tf-idf values on these sentences
bm25_encoder.fit(senetences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("/content/bm25_values.json")


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    index=index)

In [29]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7c9e420399c0>, index=<pinecone.data.index.Index object at 0x7c9f6e5352d0>)

In [30]:
retriever.add_texts(
    texts=["Artificial intelligence is transforming industries.",
    "Pinecone provides vector database solutions.",
    "Hybrid search combines semantic and keyword search."])

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
retriever.invoke("What is Hybrid Search?")

[Document(metadata={'score': 0.603588581}, page_content='Hybrid search combines semantic and keyword search.'),
 Document(metadata={'score': 0.117051736}, page_content='Pinecone provides vector database solutions.'),
 Document(metadata={'score': 0.0879552066}, page_content='Artificial intelligence is transforming industries.')]

In [32]:
retriever.invoke("What is AI?")

[Document(metadata={'score': 0.284713715}, page_content='Artificial intelligence is transforming industries.'),
 Document(metadata={'score': 0.0795862824}, page_content='Hybrid search combines semantic and keyword search.'),
 Document(metadata={'score': 0.0588818714}, page_content='Pinecone provides vector database solutions.')]